<a href="https://colab.research.google.com/github/BastienCherel/TripAdvisor-Recommendation-Challenge/blob/main/NLP_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing  : First Project
## TripAdvisor Recommendation Challenge
Beating BM25


### Installing BM25

In [4]:
!pip install rank_bm25
!pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


### Download data

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/hotel-reviews")

print("Path to dataset files:", path)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 717M/717M [00:19<00:00, 38.1MB/s] 

Extracting files...


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/joebeachcapital/hotel-reviews/versions/2



### Data Preprocessing

#### Hotels

In [6]:
import pandas as pd

data_hotels = pd.read_csv(path + '/offerings.csv')

data_hotels = data_hotels.drop(columns=["phone", "details","region_id","type", "url"])
import json
import re

# Function to fix the JSON format, while preserving apostrophes in words
def fix_json_format(address_string):


    # 3. Detect all apostrophes inside double quotes and remove them
    # This will remove any single quote (apostrophe) between a pair of double quotes
    address_string = re.sub(r'("(?:[^"\\]|\\.)*?)\'(.*?")', r'\1\2', address_string)


    # 4. Handle any double quotes inside a street address properly (if needed)
    # Escape problematic double quotes inside actual string values like street names
    address_string = re.sub(r'(?<!\\)"([A-Za-z ]*)"(?=\s+Street)', r'\"\1\"', address_string)
    address_string = re.sub(r"'", r'"', address_string)

    return address_string

# Function to fix and convert address strings in the DataFrame
def convert_address_string(address_string):
    if isinstance(address_string, str):
        fixed_string = fix_json_format(address_string)
        return json.loads(fixed_string)  # Convert to dictionary
    return address_string  # Return as is if not a string

# Apply the function to the 'address' column
data_hotels['address'] = data_hotels['address'].apply(convert_address_string)
# Convert the 'address' column into a dataframe where each key becomes a new column
address_df = pd.json_normalize(data_hotels['address'])

# Now, concatenate this new dataframe to your original dataframe
data_hotels = pd.concat([data_hotels, address_df], axis=1)

# Optionally, you can drop the original 'address' column if you no longer need it
data_hotels.drop(columns=['address'], inplace=True)

print(data_hotels.head())

   hotel_class      id                                             name  \
0          4.0  113317                    Casablanca Hotel Times Square   
1          5.0   76049  Four Seasons Hotel Los Angeles at Beverly Hills   
2          3.5   99352                   Hilton Garden Inn Times Square   
3          4.0   93589                           The Michelangelo Hotel   
4          4.0  217616                          The Muse Hotel New York   

  region        street-address postal-code       locality  
0     NY  147 West 43rd Street       10036  New York City  
1     CA       300 S Doheny Dr       90048    Los Angeles  
2     NY     790 Eighth Avenue       10019  New York City  
3     NY  152 West 51st Street       10019  New York City  
4     NY  130 West 46th Street       10036  New York City  


#### Reviews

In [7]:
data_reviews = pd.read_csv(path + '/reviews.csv')

data_reviews.drop(columns=["id", "via_mobile","author","date", "date_stayed","num_helpful_votes", "title"], inplace = True)

def fix_json_format(address_string):
    address_string = re.sub(r"'", r'"', address_string)
    return address_string

#  Function to fix and convert address strings in the DataFrame
def convert_review_string(address_string):
    if isinstance(address_string, str):
        fixed_string = fix_json_format(address_string)
        return json.loads(fixed_string)  # Convert to dictionary
    return address_string  # Return as is if not a string

# Apply the function to the 'address' column
data_reviews['ratings'] = data_reviews['ratings'].apply(convert_review_string)


# Convert the 'address' column into a dataframe where each key becomes a new column
reviews_df = pd.json_normalize(data_reviews['ratings'])

# Now, concatenate this new dataframe to your original dataframe
data_reviews = pd.concat([data_reviews, reviews_df], axis=1)

# Optionally, you can drop the original 'address' column if you no longer need it
data_reviews.drop(columns=['ratings','check_in_front_desk', 'business_service_(e_g_internet_access)'], inplace=True)

import numpy as np
# Calculate the average rating based on the specified columns
data_reviews["rating"] = data_reviews[[
    "service", "cleanliness", "overall", "value",
    "location", "sleep_quality", "rooms"
]].mean(axis=1)

data_reviews.drop(columns=["service", "cleanliness", "overall", "value",
    "location", "sleep_quality", "rooms"] , inplace=True)

print(data_reviews.head())

                                                text  offering_id    rating
0  Stayed in a king suite for 11 nights and yes i...        93338  5.000000
1  On every visit to NYC, the Hotel Beacon is the...        93338  5.000000
2  This is a great property in Midtown. We two di...      1762573  4.285714
3  The Andaz is a nice hotel in a central locatio...      1762573  4.857143
4  I have stayed at each of the US Andaz properti...      1762573  4.428571


### Datasets merging

In [8]:
# Rename columns from 'hotel' table with 'hotel_' prefix
hotel_columns = {col: f'hotel_{col}' for col in data_hotels.columns if col != 'offering_id'}

# Rename columns from 'reviews' table with 'reviews_' prefix
reviews_columns = {col: f'reviews_{col}' for col in data_reviews.columns if col != 'id'}

data = pd.merge(data_reviews, data_hotels, left_on="offering_id", right_on="id", how="left",suffixes=("_rewiew", "_hotel"))
data = data.drop(columns=["offering_id"])

data = data.rename(columns={**hotel_columns, **reviews_columns})
data = data.dropna()

print(data.head())

                                        reviews_text  reviews_rating  \
0  Stayed in a king suite for 11 nights and yes i...        5.000000   
1  On every visit to NYC, the Hotel Beacon is the...        5.000000   
2  This is a great property in Midtown. We two di...        4.285714   
3  The Andaz is a nice hotel in a central locatio...        4.857143   
4  I have stayed at each of the US Andaz properti...        4.428571   

   hotel_hotel_class  hotel_id        hotel_name hotel_region  \
0                3.0     93338      Hotel Beacon           NY   
1                3.0     93338      Hotel Beacon           NY   
2                4.0   1762573  Andaz 5th Avenue           NY   
3                4.0   1762573  Andaz 5th Avenue           NY   
4                4.0   1762573  Andaz 5th Avenue           NY   

           hotel_street-address hotel_postal-code hotel_locality  
0  2130 Broadway at 75th Street             10023  New York City  
1  2130 Broadway at 75th Street           

### BM vs. My model

In [9]:
from rank_bm25 import BM25Okapi

class Query:
    def __init__(self, text, avg_rating, region	, locality):
        self.text = text
        self.rating = avg_rating
        self.region = region
        self.locality = locality

    def bm25_model(self, reviews):
        filtered_reviews = reviews[(reviews['hotel_region'] == self.region) & (reviews['hotel_locality'] == self.locality)]

        corpus = filtered_reviews["reviews_text"]
        tokenized_corpus = [rating.split(" ") for rating in corpus]

        bm25 = BM25Okapi(tokenized_corpus)
        tokenized_query = self.text.split(" ")
        doc_scores = bm25.get_scores(tokenized_query)
        most_related_place_bm = filtered_reviews.iloc[doc_scores.argmax()]["hotel_id"]
        return most_related_place_bm

    def my_model(self, reviews):
        filtered_reviews = reviews[(reviews['hotel_region'] == self.region) & (reviews['hotel_locality'] == self.locality)]

        corpus = filtered_reviews["reviews_text"][0:5]
        tokenized_corpus = [rating.split(" ") for rating in corpus]
        corpus = filtered_reviews["reviews_text"]
        tokenized_corpus = [rating.split(" ") for rating in corpus]

        bm25 = BM25Okapi(tokenized_corpus)
        tokenized_query = self.text.split(" ")
        doc_scores = bm25.get_scores(tokenized_query)
        most_related_place_bm = filtered_reviews.iloc[doc_scores.argmax()]["hotel_id"]
        return most_related_place_bm

    def evaluate(self, selected_place, reviews):
        filtered_reviews = reviews[reviews['hotel_id'] == selected_place]
        mse = 0
        for i in range(len(filtered_reviews)):
            review = filtered_reviews.iloc[i]
            mse += (self.rating - review["reviews_rating"]) **2
        mse /= filtered_reviews.shape[0]
        return mse


query = Query(text="best breakfast",
              avg_rating = 4,
              region="MA",
              locality = "Boston")


similates_id = query.bm25_model(data)
print("Selected hotel:" , np.unique(data[data['hotel_id'] == similates_id][["hotel_name", "hotel_region", "hotel_locality"]]))
mse = query.evaluate(similates_id,data)
print("Mean Squared Error - BM25:", mse)

similates_id = query.my_model(data)
print("Selected hotel:" , np.unique(data[data['hotel_id'] == similates_id][["hotel_name", "hotel_region", "hotel_locality"]]))
mse = query.evaluate(similates_id,data)
print("Mean Squared Error - My model:", mse)

Selected hotel: ['Boston' 'MA' 'Sheraton Boston Hotel']
Mean Squared Error - BM25: 0.7810000750680801
Selected hotel: ['Boston' 'MA' 'Sheraton Boston Hotel']
Mean Squared Error - My model: 0.7810000750680801
